<a href="https://colab.research.google.com/github/danangwijaya/JLH_Air/blob/main/jlh_air_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INISIASI LIBRARY YANG DIGUNAKAN
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime
import fiona
fiona.supported_drivers

# import glob
# import os
# # import leafmap

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [ ]:
#INISIASI DATA YANG DIGUNAKAN
skor_pl = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing/skor_pl.csv')
skor_kba = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing/skor_kba.csv')
skor_kva = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing/skor_kva.csv')
data_kba = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/KBA/KBA_Bali.shp')
data_kva = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/KVA/KVA_Bali.shp')
data_pl = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/PL/PL_Bali.shp')

# MENYAMAKAN SISTEM KOORDINAT
kba = data_kba.to_crs(3395)
kva = data_kva.to_crs(3395)
pl = data_pl.to_crs(3395)

In [ ]:
#PROSES OVERLAY DATA
overlay_ekoregion = kba.overlay(kva, how='intersection')
overlay_pl = overlay_ekoregion.overlay(pl, how='intersection')
jlh = gpd.GeoDataFrame(overlay_pl)

#PROSES JOIN TABLE SKOR KBA, KVA, dan PL
join_kba = pd.merge(jlh, skor_kba, left_on='KBA_250', right_on='KBA_250')
join_kva = pd.merge(join_kba, skor_kva, left_on='KVA_250', right_on='KVA_250')
jlh = pd.merge(join_kva, skor_pl, left_on='pl2021_id', right_on='kodepl')

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1803: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:2451: UserWarning: `keep_geom_type=True` in overlay resulted in 247 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [ ]:
#MENGHITUNG SKOR JASLING
jlh['SPgAir'] = (jlh['KBA_Pg']*0.28) + (jlh['KVA_Pg']*0.12) + (jlh['PL_Pg']*0.6)
jlh['SPyAir'] = (jlh['KBA_Py']*0.28) + (jlh['KVA_Py']*0.12) + (jlh['PL_Py']*0.6)

#KLASIFIKASI SKOR JASLING(Sangat Rendah - Sangat Tinggi)
class_pg = [
    (jlh['SPgAir'] <= 1.8),
    (jlh['SPgAir'] > 1.8) & (jlh['SPgAir'] <= 2.6),
    (jlh['SPgAir'] > 2.6) & (jlh['SPgAir'] <= 3.4),
    (jlh['SPgAir'] > 3.4) & (jlh['SPgAir'] <= 4.2),
    (jlh['SPgAir'] > 4.2)
    ]

# class_py = [
#     (jlh['SPyAir'] <= 1.8),
#     (jlh['SPyAir'] > 1.8) & (jlh['SPyAir'] <= 2.6),
#     (jlh['SPyAir'] > 2.6) & (jlh['SPyAir'] <= 3.4),
#     (jlh['SPyAir'] > 3.4) & (jlh['SPyAir'] <= 4.2),
#     (jlh['SPyAir'] > 4.2)
#     ]

class_py = class_pg

values = ['Sangat Rendah', 'Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi']


jlh['KPgAir'] = np.select(class_pg, values)
jlh['KPyAir'] = np.select(class_py, values)

jlh.head()

In [ ]:
# VISUALISASI HASIL FILE

import matplotlib
import matplotlib.pyplot as plt
from mapclassify import classify

jlh.explore("KPgAir", cmap="RdYlGn")

In [ ]:
# MENYIMPAN HASIL FILE
now = datetime.now()
date = now.strftime("%d%m%Y%H%M%S")
url_file = '/content/drive/MyDrive/Colab Notebooks/result/Jasling_'+date+'.gpkg'
jlh.to_file(url_file, driver='GPKG', layer='JLH')